# Menyambungkan Colab dengan Cloud

In [3]:
from google.colab import auth    # import autentikasi dari google colab
auth.authenticate_user()         # autentikasi user google
print('Authenticated')           # jika berhasil akan muncul kalimat 'Authenticated'

from google.cloud import bigquery             # import bigquery dari google cloud
project_id = "hacktiv8-farhan-1"              # nama project id dari google cloud, pada bagian 'select a project', kolom 'ID'
client = bigquery.Client(project=project_id)  # membuat variabel client dari bigquery agar mempersingkat

Authenticated


# Problem Statement
Menganalisa waktu terselesaikannya tindak kejahatan `Theft` yang merupakan tindak kejahatan paling banyak pada Kota Austin sepanjang tahun 2016

Smart Analysis:
1. Specific : Kejahatan `Theft` pada kota Austin
2. Measurable = Bulan apa kejahatan `Theft` tertinggi
3. Achieveable = Perlu dilakukannya perbandingan kejahatan `Theft` dengan kejahatan lainnya
4. Relevant = Perbandingan waktu menyelesaikan kasus dengan waktu dilaporkan
5. Time-Bound = Kejahatan `Theft` sepanjang tahun 2016

# Penjabaran Masalah

## Menampilkan tabel crime pada 5 baris pertama

In [7]:
P0LC3_Farhan_1 = client.query ('''
SELECT *                                        # Mengambil semua kolom yang ada
FROM bigquery-public-data.austin_crime.crime    # Tabel asal BigQuery
LIMIT 5                                         # Dibatasi hanya 5 yang muncul
''').to_dataframe()


P0LC3_Farhan_1

,unique_key,address,census_tract,clearance_date,clearance_status,council_district_code,description,district,latitude,longitude,location,location_description,primary_type,timestamp,x_coordinate,y_coordinate,year,zipcode
0,2015150483,"Austin, TX",NaN,2015-01-27 12:00:00+00:00,Not cleared,<NA>,RAPE,B,NaN,NaN,None,nan,Rape,2015-01-15 12:00:00+00:00,<NA>,<NA>,2015,None
1,20151542027,"Austin, TX",NaN,2015-07-24 12:00:00+00:00,Not cleared,<NA>,RAPE,D,NaN,NaN,None,nan,Rape,2015-06-03 12:00:00+00:00,<NA>,<NA>,2015,None
2,20152001758,"Austin, TX",NaN,2015-08-15 12:00:00+00:00,Not cleared,<NA>,RAPE,F,NaN,NaN,None,nan,Rape,2015-07-19 12:00:00+00:00,<NA>,<NA>,2015,None
3,20151340282,"Austin, TX",NaN,2015-06-15 12:00:00+00:00,Cleared by Exception,<NA>,RAPE,UK,NaN,NaN,None,nan,Rape,2015-05-14 12:00:00+00:00,<NA>,<NA>,2015,None
4,20151871268,"Austin, TX",NaN,2015-10-27 12:00:00+00:00,Not cleared,<NA>,RAPE,UK,NaN,NaN,None,nan,Rape,2015-07-06 12:00:00+00:00,<NA>,<NA>,2015,None


## Apa saja jenis tindak kejahatan yang terjadi di kota Austin selama tahun 2016?

Di kota Austin, sepanjang tahun 2016 terdapat kejahatan
- Theft
- Burglary
- Agg Assault
- Auto Theft
- Robbery
- Rape
- Murder

dan kejahatan yang tertinggi merupakan `Theft` dimana terjadi 26240 kasus

In [57]:
P0LC3_Farhan_2 = client.query ('''
SELECT
        DISTINCT primary_type,                                 # Mengfilter primary_type sehingga tidak ada perulangan dengan isi yang sama
        Count (unique_key)                                     # Menghitung kasus, dimana unique_key merupakan kasus dengan angka yang berbeda, sehingga bisa digunakan untuk menghitung jumlah kasus
        Over (Partition By primary_type) as Count_Cases        # Menghitung hanya berdasarkan primary_type
FROM bigquery-public-data.austin_crime.crime                   # Tabel asal di BigQuery
WHERE year = 2016                                              # Yang dianalisa hanya pada tahun 2016
ORDER BY Count_Cases DESC                                      # Diurutkan berdasarkan Count_Cases terbanyak
''').to_dataframe()

P0LC3_Farhan_2

,primary_type,Count_Cases
0,Theft,26240
1,Burglary,5171
2,Agg Assault,2177
3,Auto Theft,2025
4,Robbery,1050
5,Rape,759
6,Murder,38


## Berapa banyak kasus tindak kejahatan berdasarkan kasus pembersihannya (clearance status) selama tahun 2016?

Di kota Austin, sepanjang tahun 2016 yang paling banyak adalah belum terselesaikan yaitu sebanyak 29715 kasus

In [22]:
P0LC3_Farhan_3 = client.query ('''
SELECT
      DISTINCT Clearance_Status,                             # Mengfilter Clearance_Status sehingga tidak ada perulangan dengan isi yang sama
      Count (unique_key)                                     # Menghitung kasus, dimana unique_key merupakan kasus dengan angka yang berbeda, sehingga bisa digunakan untuk menghitung jumlah kasus
      Over (Partition By clearance_status) as Count_Status   # Menghitung hanya berdasarkan primary_type
FROM
      bigquery-public-data.austin_crime.crime                # Tabel asal BigQuery
WHERE
      year = 2016                                            # Hanya pada tahun 2016
ORDER BY
      Count_Status DESC                                      # Diurutkan berdasarkan Count_Status yang terbesar
''').to_dataframe()

P0LC3_Farhan_3

,Clearance_Status,Count_Status
0,Not cleared,29715
1,Cleared by Arrest,5012
2,None,1719
3,Cleared by Exception,1014


## Jenis kejahatan apa saja yang paling banyak belum terselesaikan kasusnya di tahun 2016?

Di Tahun 2016 pada Kota Austin, kejahatan yang paling banyak belum diselesaikan adalah `Theft` sebanyak 21894 kasus

In [58]:
P0LC3_Farhan_4 = client.query ('''
SELECT
        Distinct primary_type,                              # Mengfilter primary_type sehingga tidak ada perulangan dengan isi yang sama
        Count (unique_key)                                  # Menghitung kasus, dimana unique_key merupakan kasus dengan angka yang berbeda, sehingga bisa digunakan untuk menghitung jumlah kasus
        Over (Partition By primary_type) as Count_Cases     # Menghitung hanya berdasarkan primary_type
FROM
        bigquery-public-data.austin_crime.crime             # Tabel asal BigQuery
WHERE
        year = 2016                                         # Hanya pada tahun 2016
        AND
        clearance_status = 'Not cleared'                    # clearase_status hanya yang berisi 'Not cleared'
ORDER BY
        Count_Cases DESC                                    # Diurutkan berdasarkan Count_Cases yang terbesar

''').to_dataframe()

P0LC3_Farhan_4

,primary_type,Count_Cases
0,Theft,21894
1,Burglary,4440
2,Auto Theft,1540
3,Agg Assault,919
4,Robbery,560
5,Rape,361
6,Murder,1


## Nomor 5

Di bulan apa di tahun 2016 kasus pencurian (theft) sangat banyak terjadi?

In [36]:
P0LC3_Farhan_5 = client.query('''
SELECT
      EXTRACT(MONTH FROM timestamp) AS Bulan,   # Mengekstrak bulan dari timestamp
      COUNT(primary_type) AS Cases_Count        # Menghitung jumlah primary_type yang terjadi
FROM
      bigquery-public-data.austin_crime.crime   # Asal tabel BigQuery
WHERE
      primary_type = 'Theft' AND year = 2016    # Hanya berisi 'Theft' pada kolom primary_type dan hanya pada tahun 2016
GROUP BY
      Bulan                                     # Digabungkan berdasarkan bulan
ORDER BY
      Cases_Count DESC                          # Diurutkan berdasarkan Count_Cases yang terbesar
''').to_dataframe()

P0LC3_Farhan_5

,Bulan,Cases_Count
0,4,2282
1,3,2271
2,6,2249
3,10,2235
4,1,2213
5,8,2204
6,5,2183
7,7,2163
8,12,2162
9,9,2156


## Di distrik apa yang paling banyak terjadi kasus pencurian selama 2016?

Di Austin pada Tahun 2016, distrik yang paling banyak terjadi kasus pencarian adalah pada district `D` sebanyak 4117 kasus, dan yang paling sedikit pada district `88` yang hanya 13 kasus.

In [39]:
P0LC3_Farhan_6 = client.query('''
SELECT *
FROM (
      SELECT
            district,                                   # Mengambil kolom district
            COUNT(primary_type) AS Cases_Count          # Menghitung jumlah primary_type yang terjadi
      FROM
            bigquery-public-data.austin_crime.crime     # Asal tabel BigQuery
      WHERE
            primary_type = 'Theft' AND year = 2016      # Hanya berisi 'Theft' pada kolom primary_type dan hanya pada tahun 2016
      GROUP BY
            district                                    # Digrupkan berdasarkan district
)
ORDER BY
      Cases_Count DESC                                  # Diurutkan berdasarkan Count_Cases yang terbesar
''').to_dataframe()

P0LC3_Farhan_6

,district,Cases_Count
0,D,4117
1,B,3330
2,A,3096
3,F,3034
4,E,2982
5,I,2870
6,H,2734
7,G,2117
8,C,1724
9,AP,143


## Berapa lama rata-rata waktu (dalam hari) kasus tindak kejahatan 'theft' terselesaikan sejak kasus tersebut terangkat selama tahun 2016?

Pada tahun 2016, Kota Austin menyelesaikan kasus tindak kejatahan `Theft` rata-rata 14 Hari dari awal dilaporkan tindak kejahatan

In [60]:
P0LC3_Farhan_7 = client.query('''
SELECT
            AVG(DATE_DIFF(clearance_date, timestamp, DAY)) AS Average_Duration  # Membuat rata-rata perbedaan antara clearance_date dengan timestamp dihitung hari
FROM (
      SELECT
            timestamp,                                                          # Mengambil kolom timestamp
            clearance_date                                                      # Mengambil kolom clearance_date
      FROM
            bigquery-public-data.austin_crime.crime                             # Asal tabel BigQuery
      WHERE
            primary_type = 'Theft' AND                                          # Berisi hanya primary_type 'Theft'
            year = 2016 AND                                                     # Berisi hanya tahun 2016
            timestamp IS NOT NULL AND                                           # Timestamp tidak null
            clearance_date IS NOT NULL AND                                      # clearance_date tidak null
            clearance_status IS NOT NULL AND                                    # clearance_status tidak null
            clearance_status != 'Not cleared'                                   # clearance_status bukan berisi 'Not cleared'
)

''').to_dataframe()

P0LC3_Farhan_7

,Average_Duration
0,14.295328


# Kesimpulan

Di Kota Austin pada Tahun 2016, kejahatan yang paling tinggi merupakan kejahatan `Theft` dan merupakan kasus yang paling banyak belum diselesaikan. Bulan April adalah kasus paling tinggi dilakukannya kejahatan tersebut dibandingkan dengan bulan-bulan lainnya, dan yang kasus terbanyak terjadi pada district `D`, dari kejahatan tersebut rata-rata waktu yang dibutuhkan untuk menyelesaikan kasus adalah 14 hari.